###  Coding Challenge - Getting Started

<img align="right" style="max-width: 200px; height: auto" src="https://github.com/sandrobl/ml-eurosat/blob/main/hsg_logo.png?raw=1">
<img align="center" style="max-width: 300px; height: auto" src="https://github.com/sandrobl/ml-eurosat/blob/main/sentinel2.jpg?raw=1">

8,860,1.00 MCS Machine Learning, Spring Term 2025, University of St.Gallen (HSG)

The lab environment of the **8,860,1.00 Machine Learning** course is powered by Jupyter Notebooks (https://jupyter.org), which allows one to perform a great deal of data analysis and statistical validation. In this first lab, we want to touch on the basic concepts and techniques of such notebooks. Furthermore, its capabilities will be demonstrated based on a few simple and introductory examples.

### Objectives:

With the help of this notebook you should be able to:
    
> 1. Understand the basic funcitonality of the rasterio framework
> 2. Apply rasterio to load GTiff data
> 3. Visualize multi-band satellite imagery
> 4. Perform basic band arithmetic to compute the normalized difference vegetation index (NDVI)
> 5. Load and display samples from the challenge testset

## 2. Setup of the Jupyter Notebook Environment

Similar to the previous labs, we need to import a couple of Python libraries that allow for data analysis and data visualization. We will use `os` and `glob` to collect the filepaths for the data samples, `numpy` for array operations, `matplotlib` to display images, and `rasterio` to handle raster data. You can find the documentation of the `rasterio` library with an overview of its functionality [here](https://rasterio.readthedocs.io).

In [7]:
%pip install rasterio
%pip install matplotlib

import os
import glob
import numpy as np
import rasterio as rio
from rasterio.plot import reshape_as_image
import matplotlib.pyplot as plt
import shutil

%matplotlib inline

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:

zip_file_path = "EuroSATallBands.zip"

if not os.path.exists(zip_file_path):
    print(f"File '{zip_file_path}' not found. Downloading...")
    !powershell Invoke-WebRequest -Uri "https://madm.dfki.de/files/sentinel/EuroSATallBands.zip" -OutFile "EuroSATallBands.zip"

    # Alternative download command for Linux & MacOS
    #!wget --no-check-certificate https://madm.dfki.de/files/sentinel/EuroSATallBands.zip

else:
    print(f"File '{zip_file_path}' already exists. Skipping download.")



File 'EuroSATallBands.zip' already exists. Skipping download.


## EuroSat Data Loading

First, let's collect all the files that we downloaded

In [9]:
zip_file_path = "EuroSATallBands.zip"
extraction_path = "./data"

# Check if the extraction path exists
if not os.path.exists(extraction_path):
    print(f"Unpacked archive to {extraction_path}")
    shutil.unpack_archive(zip_file_path, extraction_path)
else:
    print(f"Folder already exists at {extraction_path}")

# change this to your eurosat path
eurosat_dir = "./data/ds/images/remote_sensing/otherDatasets/sentinel_2/tif"

Folder already exists at ./data


In [10]:
samples = glob.glob(os.path.join(eurosat_dir, "*", "*.tif"))
len(samples)

27000

We have 27,000 files across 10 classes.



# Testset Data Loading

The testset has a slightly different structure than Eurosat. There are no labels and the data is stored in `numpy` `.npy` instead of GTiff.

First, we have to download the data from [Kaggle](https://www.kaggle.com/competitions/8-860-1-00-coding-challenge-2025/data).

In [11]:
zip_test_file_path = "./testset.zip"
extraction_path = "./data"

testset_dir = os.path.join(extraction_path, "testset")

if not os.path.exists(zip_test_file_path):
    print(f"File '{zip_test_file_path}' not found. Will download the testdata here")
    !wget https://github.com/sandrobl/ml-eurosat/raw/main/testset.zip

    #Windows Powershell version
    #!powershell Invoke-WebRequest -Uri "https://github.com/sandrobl/ml-eurosat/raw/main/testset.zip" -OutFile "testset.zip"

else:
    print(f"File '{zip_test_file_path}' already exists. Skipping download.")


# Check if the testset directory exists
if not os.path.exists(testset_dir):
    print(f"Unpacked archive to {extraction_path}")
    shutil.unpack_archive(zip_test_file_path, extraction_path)
else:
    print(f"Testset folder already exists at {testset_dir}")


test_samples = glob.glob(os.path.join(testset_dir, "*.npy"))
len(test_samples)

File './testset.zip' already exists. Skipping download.
Testset folder already exists at ./data\testset


4232

## Next steps

Create a (deep learning) model to predict the most likely Eurosat class for each image of the testset. Think about creating the dataset class and data-loader for training, possible model architectures, and perhaps even how to best address the shift between train and test data.

In [ ]:
# Install necessary libraries
%pip install torch torchvision torchsummary pandas scikit-learn


import rasterio  # Required for .tif images
import datetime
import os
import numpy as np
import pandas as pd
import rasterio as rio
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.decomposition import PCA

if __name__ == '__main__':

    # ------------------------
    # 1. Define Dataset Class
    # ------------------------
    classes = [
        "AnnualCrop", "Forest", "HerbaceousVegetation", "Highway",
        "Industrial", "Pasture", "PermanentCrop", "Residential",
        "River", "SeaLake",
    ]

    BASE_DIR = os.getcwd() 


    def normalize_for_display(band_data):
        band_data = np.array(band_data, dtype=np.float32) / 65535.0  # Normalize to 0-1
        mean = np.mean(band_data, axis=(0, 1))
        std = np.std(band_data, axis=(0, 1))
        return (band_data - mean) / (std + 1e-7)


    class EurostatDataset(Dataset):
        def __init__(self, root_dir, labels_csv=None, transform=None, file_type="tif"):
            self.root_dir = root_dir
            self.transform = transform
            self.file_type = file_type  # "tif" for training, "npy" for testing
            self.images = []
            self.labels = []
            self.class_to_idx = {}  # Class label mapping

            if labels_csv:
                # **For TEST SET (Uses CSV for labels)**
                self.labels_df = pd.read_csv(labels_csv)
                self.class_to_idx = {cls: idx for idx, cls in enumerate(sorted(self.labels_df['label'].unique()))}

                for _, row in self.labels_df.iterrows():
                    file_name = f"test_{row['test_id']}.npy"
                    file_path = os.path.join(root_dir, file_name)
                    if os.path.exists(file_path):
                        self.images.append(file_path)
                        self.labels.append(self.class_to_idx[row['label']])
            else:
                # **For TRAIN SET (Auto-labels based on folder names)**
                folders = sorted(os.listdir(root_dir))
                self.class_to_idx = {folder: idx for idx, folder in enumerate(folders)}

                for folder in folders:
                    folder_path = os.path.join(root_dir, folder)
                    if os.path.isdir(folder_path):
                        for file in os.listdir(folder_path):
                            if file.endswith(f".{file_type}"):  # Match file type
                                self.images.append(os.path.join(folder_path, file))
                                self.labels.append(self.class_to_idx[folder])

        def __len__(self):
            return len(self.images)

        def __getitem__(self, idx):
            img_path = self.images[idx]  
            label = self.labels[idx]

            if self.file_type == "tif":
                # **Load .tif for training**
                with rasterio.open(img_path) as src:
                    image = src.read([1, 2, 3])  # Extract RGB bands
                    image = np.moveaxis(image, 0, -1)  # Convert from (C, H, W) → (H, W, C)
            else:
                # **Load .npy for testing**
                image = np.load(img_path)[:, :, :3]  # Extract first 3 bands (RGB)

            # Normalize: Convert to float32 and scale to [0, 255]
            image = (image / image.max()) * 255.0  
            image = image.astype(np.uint8)

            # Convert to PIL image
            image = Image.fromarray(image)

            # Apply transformations
            if self.transform:
                image = self.transform(image)

            return image, label


    # ------------------------
    # 2. Define Transformations
    # ------------------------
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # ------------------------
    # 3. Load Dataset & DataLoader
    # ------------------------
    test_dataset = EurostatDataset(
        
        root_dir=os.path.join(BASE_DIR, "data", "testset"),
        labels_csv=os.path.join(BASE_DIR, "test_labels.csv"),
        transform=transform,
        file_type="npy"
    )

    print(f"Total samples in test_dataset: {len(test_dataset)}")

    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)

    print(f"Total number of batches: {len(test_loader)}")
    print(f"Total samples: {len(test_loader.dataset)}")

    # ------------------------
    # 4. Define Model
    # ------------------------
    class EurosatModel(nn.Module):
        def __init__(self, num_classes=10):
            super(EurosatModel, self).__init__()
            self.model = models.resnet50(pretrained=True)
            self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

        def forward(self, x):
            return self.model(x)

    model = EurosatModel(num_classes=10)

    # ------------------------
    # 5. Training Setup
    # ------------------------
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

    # ------------------------
    # 6. Training Loop
    # ------------------------

    train_dataset = EurostatDataset(
        root_dir=os.path.join(BASE_DIR, "data", "ds", "images", "remote_sensing", "otherDatasets", "sentinel_2", "tif"),
        transform=transform,
        file_type="tif"
    )

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0, pin_memory=True)

    num_epochs = 20
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 10 == 9:
                print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 10:.4f}')
                running_loss = 0.0

        scheduler.step()

    print('Finished Training')

    # ------------------------
    # 7. Evaluation Loop (Fixed)
    # ------------------------
    model.to(device)
    model.eval()  # Set model to evaluation mode

    correct = 0
    total = 0
    predictions = []  # Store predictions

    with torch.no_grad():  
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            predictions.extend(predicted.cpu().numpy())

    print(f"First 10 test predictions: {predictions[:10]}")
    print(f'Test Accuracy: {100 * correct / total:.2f}%')

    accuracy = 100 * correct / total

    print(f"First 10 test predictions: {predictions[:10]}")
    print(f'Test Accuracy: {100 * correct / total:.2f}%')

    # Save model
    current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    # Create a filename with the current date, time, and accuracy
    filename = f"model_checkpoint_accuracy_{accuracy:.2f}_{current_time}.pth"

    torch.save(model.state_dict(), os.path.join("./trained_models", filename))




[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\sandr\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sandr\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Note: you may need to restart the kernel to use updated packages.
Total samples in test_dataset: 4232
Total number of batches: 67
Total samples: 4232
Using device: cuda
Epoch [1/20], Step [10/422], Loss: 1.3677


KeyboardInterrupt: 